In [1]:
import sys
import commonutils
from commonutils import ModelResults

import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
#from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from dataclasses import dataclass
import prettyprinter as pp

from sklearn.cross_decomposition import PLSRegression
import warnings
import sys

from sklearn import preprocessing

from copy import deepcopy
import pickle

import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import sys
sys.path.append("./CLossLr")
import customlosslr as clr

from commonutils import ModelsStore
warnings.simplefilter("ignore")

from commonfuncsforcli import *

In [ ]:
equations = {"Te": "Kinetic_Energy", \
         "V_NN": "Nuclear_Repulsion",\
         "V_eN": "One_Electron_Energy - Kinetic_Energy",\
         "EX": "EX",\
         "E_J": "Two_Electron_Energy - EX - EC",\
         "DC": "Dispersion_correction",\
         "EC": "EC"}
SHIFTFT = "DC"

selected_basisset = "MINIX"
selected_functional = "PBE"
functionals = []
basis_sets = []

print("Selected functional: ", selected_functional)
print(" Selected basis set: ", selected_basisset)
print("        Functionals: ", functionals)
print("         Basis sets: ", basis_sets)

featuresvalues_perset,\
fullsetnames, \
models_results, \
supersetnames = readdata(shiftusingFT=SHIFTFT, \
                selected_functionalin=selected_functional, \
                selected_basisin=selected_basisset, \
                equations=equations)

sep = "_"
for setname in fullsetnames:
    desciptors = {}
    k = selected_functional + sep +\
                selected_basisset 
    for features in featuresvalues_perset[setname]:
        for val in features:
            if val.find(k) != -1:
                if val not in desciptors:
                    desciptors[val] = [features[val]]
                else:
                    desciptors[val].append(features[val])

    for features in featuresvalues_perset[setname]:
        for val in features:
            for func in functionals:
                for basis in basis_sets:
                    if not(basis == selected_basisset and \
                            func == selected_functional):
                        if val.find(func + sep + basis) != -1:
                            actualk = val 
                            refk  = selected_functional + sep  + \
                                    selected_basisset + \
                                    val.replace(func + sep + basis, "")
                            newk = actualk + "_difftoref"
                            if newk not in desciptors:
                                desciptors[newk] = [features[actualk]-features[refk]]
                            else:
                                desciptors[newk].append(features[actualk]-features[refk])
    
    models_results[setname].features = desciptors

# features selection
setname = "Full"
numoffeat = len(models_results[setname].features)
print("Number of features for ", numoffeat)
for setname in fullsetnames:
    if len(models_results[setname].features) != numoffeat:
        print("Number of features for ", setname, " is different")
        sys.exit(1)

toremove = []
setname = "Full"
for k in models_results[setname].features:
    if len(set(models_results[setname].features[k])) == 1:
        toremove.append(k)
        print("Constant fatures to remove: ", k)

# remove constant values
for setname in fullsetnames:
    for k in toremove:
        del models_results[setname].features[k]

# force removing features Nuclear Repulsion difference
tormfeatname = set()
print("Removing Nuclear Repulsion differences")
for setname in fullsetnames: 
    toremove = []
    for k in models_results[setname].features:
        if k.find("NR_difftoref") != -1:
            toremove.append(k)
    for k in toremove:
        #print("Removing feature ", k)
        tormfeatname.add(k)
        del models_results[setname].features[k]

print("Removed features: ", tormfeatname)
setname = "Full"
numoffeat = len(models_results[setname].features)
print("Number of features ", numoffeat)
for setname in fullsetnames:
    if len(models_results[setname].features) != numoffeat:
        print("Number of features for ", setname, " is different")
        sys.exit(1)

In [ ]:
models_store = {}
for setname in list(supersetnames)+["Full"]:
    models_store[setname] = ModelsStore()

    print("Running models for dataset: ", setname)
    X, y, features_names = \
            commonutils.build_XY_matrix \
            (models_results[setname].features, \
            models_results[setname].labels)
    X_test = None
    X_train = None
    y_test = None
    y_train = None
    fts_test = None
    fts_train = None
    if len(models_results[setname].fts) > 0:
        X_train, X_test, y_train, y_test, fts_train, fts_test \
          = train_test_split(X, y, models_results[setname].fts,
                      test_size=0.20, random_state=42)
    else:
        X_train, X_test, y_train, y_test \
          = train_test_split(X, y, test_size=0.20, random_state=42)
    setlist = models_results[setname].setnames  
    supersetlist = models_results[setname].supersetnames
    y_true = y  
    y_test_true = y_test
    y_train_true = y_train
    fts = models_results[setname].fts

    # Linear regression model to get starting beta values
    lr_start_model = clr.custom_loss_lr (loss=clr.mean_average_error)
    lr_start_split_model = clr.custom_loss_lr (loss=clr.mean_average_error)
    try:
        lr_start_model.fit(X, y)
    except Exception as e:
        print("Error: ", e)
        lr_start_model.set_solver("Nelder-Mead")
        lr_start_model.fit(X, y)
    try:
        lr_start_split_model.fit(X_train, y_train)
    except Exception as e:
        print("Error: ", e)
        lr_start_split_model.set_solver("Nelder-Mead")
        lr_start_split_model.fit(X_train, y_train)

    # Custom Loss Linear Regression
    models_store[setname].lr_custom_model =\
            clr.custom_loss_lr (loss=clr.mean_absolute_percentage_error)
    bestastart = lr_start_model.get_beta()   
    if setname == "Full":
        bestastart = np.ones(len(bestastart))
    try:
        models_store[setname].lr_custom_model.fit(X, y, \
                beta_init_values = bestastart)
    except Exception as e:
        print("Error: ", e)
        models_store[setname].lr_custom_model.set_solver("Nelder-Mead")
        models_store[setname].lr_custom_model.fit(X, y, \
                beta_init_values = bestastart)
    y_pred_custom_lr = models_store[setname].lr_custom_model.predict(X)
    if SHIFTFT != "":
        y_true, y_pred_custom_lr = shiftbackdata (y, y_pred_custom_lr, fts)
    custom_lrrmse = 0.0
    try:
        custom_lrrmse = root_mean_squared_error(y_true, y_pred_custom_lr)
    except:
        custom_lrrmse = np.sqrt(mean_squared_error(y_true, y_pred_custom_lr))    
    custom_lrrmape = mean_absolute_percentage_error(y_true, y_pred_custom_lr)
    models_store[setname].lr_custom_model_splitted  = \
            clr.custom_loss_lr (loss=clr.mean_absolute_percentage_error)
    bestastart = lr_start_split_model.get_beta()
    try:
        models_store[setname].lr_custom_model_splitted.fit(X_train, y_train, \
                beta_init_values = bestastart)
    except Exception as e:
        print("Error: ", e)
        models_store[setname].lr_custom_model_splitted.set_solver("Nelder-Mead")
        models_store[setname].lr_custom_model_splitted.fit(X_train, y_train, \
                beta_init_values = bestastart)
    y_pred_custom_lr = models_store[setname].lr_custom_model_splitted.predict(X_test)
    if SHIFTFT != "":
        y_test_true, y_pred_custom_lr = shiftbackdata (y_test, y_pred_custom_lr, fts_test)
    custom_lrrmsetest = 0.0
    try:
        custom_lrrmsetest = root_mean_squared_error(y_test_true, y_pred_custom_lr)
    except:
        custom_lrrmsetest = np.sqrt(mean_squared_error(y_test_true, y_pred_custom_lr))
    custom_lrrmapetest = mean_absolute_percentage_error(y_test_true, y_pred_custom_lr)
    y_pred_custom_lr = models_store[setname].lr_custom_model_splitted.predict(X_train)
    if SHIFTFT != "":
        y_train_true, y_pred_custom_lr = shiftbackdata (y_train, y_pred_custom_lr, fts_train)
    custom_lrrmsetrain = 0.0
    try:
        custom_lrrmsetrain = root_mean_squared_error(y_train_true, y_pred_custom_lr)
    except:
        custom_lrrmsetrain = np.sqrt(mean_squared_error(y_train_true, y_pred_custom_lr))
    custom_lrrmapetrain = mean_absolute_percentage_error(y_train_true, y_pred_custom_lr)
    print("%40s ,      Custom LR RMSE, %12.6f"%(setname,custom_lrrmse))
    print("%40s ,Custom LR Train RMSE, %12.6f"%(setname,custom_lrrmsetrain))
    print("%40s , Custom LR Test RMSE, %12.6f"%(setname,custom_lrrmsetest))
    print("%40s ,      Custom LR MAPE, %12.6f"%(setname,custom_lrrmape))
    print("%40s ,Custom LR Train MAPE, %12.6f"%(setname,custom_lrrmapetrain))
    print("%40s , Custom LR Test MAPE: %12.6f"%(setname,custom_lrrmapetest))


In [ ]:
# build RF model
classes = []
features = {}
supersetnameslist = list(supersetnames.keys())
for setname in featuresvalues_perset:
    if setname in supersetnameslist:
        print("Setname: ", setname)
        for entry in featuresvalues_perset[setname]:
            classes.append(supersetnameslist.index(setname))

X, _, features_names =\
        commonutils.build_XY_matrix (\
        models_results['Full'].features,\
        models_results['Full'].labels)
X_train, X_test, y_train, y_test = train_test_split(\
        X, classes, test_size=0.20, random_state=41)
accuracys = []
numoftrees = []
depths = []
for ntrees in range(10, 200, 10):
    for depth in [5, 10, 15, None]:
        rf = RandomForestClassifier(n_estimators=ntrees, \
                                    max_depth=depth, \
                                    random_state=42)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracys.append(accuracy)
        numoftrees.append(ntrees)
        depths.append(depth)
bestaccuracy = max(accuracys)   
bestntrees = numoftrees[accuracys.index(bestaccuracy)]
bestdepth = depths[accuracys.index(bestaccuracy)]
print("Best accuracy: ", max(accuracys), " with ", \
      bestntrees, " trees ", \
      " and depth ", bestdepth)
rf = RandomForestClassifier(n_estimators=bestntrees, \
                            max_depth=bestdepth, \
                            random_state=42)
rf.fit(X_train, y_train)
testaccuracy = rf.score(X_test, y_test)
trainaccuracy = rf.score(X_train, y_train)
overallaccuracy = rf.score(X, classes)
print("  Train accuracy: %5.2f"%(trainaccuracy))
print("   Test accuracy: %5.2f"%(testaccuracy))
print("Overall accuracy: %5.2f"%(overallaccuracy))


In [ ]:
# FLPs 
lrfullmodel = models_store["Full"].lr_custom_model
setname  = "LARGE_SYSTEMS_FLPs"
X, y, features_names =\
            commonutils.build_XY_matrix (\
            models_results[setname].features,\
            models_results[setname].labels) 

func_basis_ref  = selected_functional + "-" + selected_basisset
y_ref_func_basisset = models_results[setname].funcional_basisset_ypred[func_basis_ref]

y_true = y
fts = models_results[setname].fts
if len(fts) == 0:
    fts = [0.0]*len(y)

y_pred_rf = []
for i in range(len(y)):
    x = X[i]
    c = rf.predict([x])
    setname = supersetnameslist[c[0]]
    #print("Setname: ", setname)
    y_pred = models_store[setname].lr_custom_model.predict([x])
    y_pred_rf.append(y_pred[0])    

y_pred_full_model = lrfullmodel.predict(X)
if SHIFTFT != "":
    y_true, y_pred_full_model = shiftbackdata (y, y_pred_full_model, fts)
    _, y_pred_rf = shiftbackdata (y, y_pred_rf, fts)
fp = open("flps.csv", "w")
print(" %12s , %12s , %12s , %12s "%(\
    "True", "Full model", "RF model", func_basis_ref), \
        file=fp)
for yt, ypf, ypr, yref in zip(\
    y_true, y_pred_full_model, y_pred_rf, y_ref_func_basisset): 
    print(" %12.6f , %12.6f , %12.6f , %12.5f "%(\
        yt, ypf, ypr, yref), file=fp)
fp.close()     
mape  = mean_absolute_percentage_error(y_true, y_pred_full_model)
r2 = r2_score(y_true, y_pred_full_model)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_full_model))
print("Full model MAPE: ", mape)
print("  Full model R2: ", r2)
print("Full model RMSE: ", rmse)
mape = mean_absolute_percentage_error(y_true, y_pred_rf)
r2 = r2_score(y_true, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_rf))
print("  RF model MAPE: ", mape)
print("    RF model R2: ", r2)
print("  RF model RMSE: ", rmse)
mape = mean_absolute_percentage_error(y_true, y_ref_func_basisset)
r2 = r2_score(y_true, y_ref_func_basisset)
rmse = np.sqrt(mean_squared_error(y_true, y_ref_func_basisset))
print(func_basis_ref, " MAPE: ", mape)
print(func_basis_ref, "   R2: ", r2)
print(func_basis_ref, " RMSE: ", rmse)
# scatter plot of the results
plt.figure()
plt.scatter(y_true, y_pred_full_model, color='blue')
plt.scatter(y_true, y_pred_rf, color='red')
plt.scatter(y_true, y_ref_func_basisset, color='green')
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
plt.xlabel('True')
plt.ylabel('Predicted')
plt.title('Full model vs RF model')
plt.legend(["Full model", "RF model", func_basis_ref])
plt.show()

In [ ]:
#GMTKN55
modeltorm = "FLPs"
idexestorm = []
for i, name in enumerate(models_results["Full"].setnames):
    setname  = name.split("_")[-1]
    if setname == modeltorm:
        idexestorm.append(i)

lrfullmodel = models_store["Full"].lr_custom_model
setname  = "Full"
X, y, features_names =\
            commonutils.build_XY_matrix (\
            models_results[setname].\
            features,\
            models_results[setname].labels) 
# remove FLPs so indextorm
fts = models_results[setname].fts
if len(fts) == 0:
    fts = [0.0]*len(y)
X = np.delete(X, idexestorm, axis=0)
y = np.delete(y, idexestorm)
fts = np.delete(fts, idexestorm)

func_basis_ref  = selected_functional + "-" + selected_basisset
y_ref_func_basisset = models_results[setname].funcional_basisset_ypred[func_basis_ref]
y_ref_func_basisset = np.delete(y_ref_func_basisset, idexestorm)
y_true = y

y_pred_rf = []
for i in range(len(y)):
    x = X[i]
    c = rf.predict([x])
    setname = supersetnameslist[c[0]]
    #print("Setname: ", setname)
    y_pred = models_store[setname].lr_custom_model.predict([x])
    y_pred_rf.append(y_pred[0])    

y_pred_full_model = lrfullmodel.predict(X)
if SHIFTFT != "":
    y_true, y_pred_full_model = shiftbackdata (y, y_pred_full_model, fts)
    _, y_pred_rf = shiftbackdata (y, y_pred_rf, fts)
fp = open("predictionsgmtkn55.csv", "w")
print(" %12s , %12s , %12s , %12s "%(\
    "True", "Full model", "RF model", func_basis_ref),\
        file=fp)
for yt, ypf, ypr, yref in zip(\
    y_true, y_pred_full_model, y_pred_rf, y_ref_func_basisset): 
    print(" %12.6f , %12.6f , %12.6f , %12.5f "%(\
        yt, ypf, ypr, yref), file=fp)
fp.close()
mape  = mean_absolute_percentage_error(y_true, y_pred_full_model)
r2 = r2_score(y_true, y_pred_full_model)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_full_model))
print("Full model MAPE: ", mape)
print("  Full model R2: ", r2)
print("Full model RMSE: ", rmse)
mape = mean_absolute_percentage_error(y_true, y_pred_rf)
r2 = r2_score(y_true, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_rf))
print("  RF model MAPE: ", mape)
print("    RF model R2: ", r2)
print("  RF model RMSE: ", rmse)
mape = mean_absolute_percentage_error(y_true, y_ref_func_basisset)
r2 = r2_score(y_true, y_ref_func_basisset)
rmse = np.sqrt(mean_squared_error(y_true, y_ref_func_basisset))
print(func_basis_ref, " MAPE: ", mape)
print(func_basis_ref, "   R2: ", r2)
print(func_basis_ref, " RMSE: ", rmse)
# scatter plot of the results
plt.figure()
plt.scatter(y_true, y_pred_full_model, color='blue')
plt.scatter(y_true, y_pred_rf, color='red')
plt.scatter(y_true, y_ref_func_basisset, color='green')
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
plt.xlabel('True')
plt.ylabel('Predicted')
plt.title('Full model vs RF model')
plt.legend(["Full model", "RF model", func_basis_ref])
plt.show()

In [ ]:
#Full
lrfullmodel = models_store["Full"].lr_custom_model
setname  = "Full"
X, y, features_names =\
            commonutils.build_XY_matrix (\
            models_results[setname].features,\
            models_results[setname].labels) 
fts = models_results[setname].fts
if len(fts) == 0:
    fts = [0.0]*len(y)
func_basis_ref  = selected_functional + "-" + selected_basisset
y_ref_func_basisset = models_results[setname].funcional_basisset_ypred[func_basis_ref]
y_true = y

y_pred_rf = []
for i in range(len(y)):
    x = X[i]
    c = rf.predict([x])
    setname = supersetnameslist[c[0]]
    #print("Setname: ", setname)
    y_pred = models_store[setname].lr_custom_model.predict([x])
    y_pred_rf.append(y_pred[0])    

y_pred_full_model = lrfullmodel.predict(X)
if SHIFTFT != "":
    y_true, y_pred_full_model = shiftbackdata (y, y_pred_full_model, fts)
    _, y_pred_rf = shiftbackdata (y, y_pred_rf, fts)
mape  = mean_absolute_percentage_error(y_true, y_pred_full_model)
r2 = r2_score(y_true, y_pred_full_model)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_full_model))
print("Full model MAPE: ", mape)
print("  Full model R2: ", r2)
print("Full model RMSE: ", rmse)
mape = mean_absolute_percentage_error(y_true, y_pred_rf)
r2 = r2_score(y_true, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_rf))
print("  RF model MAPE: ", mape)
print("    RF model R2: ", r2)
print("  RF model RMSE: ", rmse)
mape = mean_absolute_percentage_error(y_true, y_ref_func_basisset)
r2 = r2_score(y_true, y_ref_func_basisset)
rmse = np.sqrt(mean_squared_error(y_true, y_ref_func_basisset))
print(func_basis_ref, " MAPE: ", mape)
print(func_basis_ref, "   R2: ", r2)
print(func_basis_ref, " RMSE: ", rmse)
# scatter plot of the results
plt.figure()
plt.scatter(y_true, y_pred_full_model, color='blue')
plt.scatter(y_true, y_pred_rf, color='red')
plt.scatter(y_true, y_ref_func_basisset, color='green')
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
plt.xlabel('True')
plt.ylabel('Predicted')
plt.title('Full model vs RF model')
plt.legend(["Full model", "RF model", func_basis_ref])
plt.show()

In [8]:
fp = open("modelscoefficients.csv", "w")    
for setname in list(supersetnames)+["Full"]:   
    lr_model = models_store[setname].lr_model
    lr_model_splitted = models_store[setname].lr_model_splitted
    lr_custom_model = models_store[setname].lr_custom_model
    lr_custom_model_splitted = models_store[setname].lr_custom_model_splitted

    X, y, features_names = \
            commonutils.build_XY_matrix (\
                models_results[setname].features, \
                models_results[setname].labels)

    # Custom LR model
    lr_test_and_rpint (lr_custom_model, X, y, setname + " Custom LR ", \
                    features_names, fp, \
                    shiftft=SHIFTFT, \
                    fts=models_results[setname].fts)
    lr_test_and_rpint (lr_custom_model_splitted, X, y, \
                    setname + " Custom LR split ", \
                    features_names, fp, \
                    shiftft=SHIFTFT, \
                    fts=models_results[setname].fts)

fp.close()